<a href="https://colab.research.google.com/github/k1151msarandega/1st-order/blob/main/1st_order_direct_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Architecture:** *1st-order model*

**Encoding Scheme:** *Direct encoding*

In [ ]:
pip install snntorch

import torch
from torchvision import datasets, transforms
import snntorch as snn

1. Define the network architecture

In [ ]:
class Net(snn.SNNModule):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = snn.Linear(28 * 28, 256)
        self.fc2 = snn.Linear(256, 64)
        self.fc3 = snn.Linear(64, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

2. Define the training and testing functions

In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()  # Clear the gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute the loss
        total_loss += loss.item() * inputs.size(0)

        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    return total_loss / len(train_loader.dataset)

def test(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
            correct += (predicted == targets).sum().item()

    accuracy = correct / len(test_loader.dataset)
    return accuracy

3. Set the random seed for reproducibility

In [ ]:
torch.manual_seed(0)

4. Define the device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


5. Define the transformation to apply to the input data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

6. Load the MNIST dataset

In [ ]:
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)


7. Create data loaders

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=64, shuffle=False, num_workers=2)

8. Initialise the network

In [ ]:
model = Net().to(device)

9. Define the loss function and optimiser

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

10. Train the model

In [ ]:
start_time = time.time()
for epoch in range(10):
    loss = train(model, train_loader, criterion, optimizer, device)
    print(f"Epoch [{epoch+1}/10], Loss: {loss:.4f}")

end_time = time.time()
training_time = end_time - start_time

11. Test the model

In [ ]:
start_time = time.time()
accuracy = test(model, test_loader, device)
end_time = time.time()
testing_time = end_time - start_time

12. Evaluation metrics

In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Testing Time: {testing_time:.2f} seconds")